In [1]:
# Import and init
from copy import deepcopy
# import logging
import os
import yaml

from pprint import PrettyPrinter
import xml.etree.ElementTree as ET
import xmltodict

from resources.parser import WeaponMetaParser, PickupsMetaParser
from resources.conversion import DLC_TO_DEVICENAME

ROOT_PATH = './vanilla'
PATH_OUTPUTS = './outputs'
DELETE_TOKEN = '!DELETE'

pp = PrettyPrinter(indent=1, compact=True, sort_dicts=False)
def pprint(obj):
    return pp.pprint(obj)

for f in ['supplementary-docs', 'merged']:
    if not os.path.isdir(f'{PATH_OUTPUTS}/{f}'):
        os.makedirs(f'{PATH_OUTPUTS}/{f}')

In [ ]:
# Crawl list of all DLCs from latest dlclist.xml.
# Devlog: 2024-01-16, GTA V 1.70, have scanned up to patch2024_02
if True:
    f = f'{ROOT_PATH}/update/update.rpf/common/data/dlclist.xml'
    dlcs_xml = ET.parse(source=f).getroot()
    dlcs = {
        'base': {
            'base': {'path': f'update/update.rpf'},
        }
    }
    for i in dlcs_xml.find('Paths'):
        dlc_archive, dlc_subpath = i.text.split(sep=':')
        
        if dlc_subpath[-1] == '/':
            dlc_subpath = dlc_subpath[0:-1]
        dlc_name = dlc_subpath.split(sep='/')[-1].lower()

        if dlc_archive == 'platform':
            dlc_path = f'x64w.rpf/dlcpacks/{dlc_name}/dlc.rpf'
        elif dlc_archive == 'dlcpacks':
            dlc_path = f'update/x64/dlcpacks/{dlc_name}/dlc.rpf'
        else:
            raise ValueError(f'Unknown archive type.')
        
        dlcs[dlc_name] = {'base': {'path': dlc_path}}
    print(f)

# Crawl list of all DLC patches from latest extratitleupdatedata.meta.
if True:
    f = f'{ROOT_PATH}/update/update.rpf/common/data/extratitleupdatedata.meta'
    dlcpatch_xml = ET.parse(source=f).getroot()
    dlcpatch_xmldict = xmltodict.parse(ET.tostring(dlcpatch_xml))
    prefix = 'update:/dlc_patch/'
    for i in dlcpatch_xmldict['SExtraTitleUpdateData']['Mounts']['Item']:
        assert i['path'].startswith(prefix) and i['path'].endswith('/')
        dlc_name = i['path'][len(prefix):-1].lower()
        dlcs[dlc_name].update({'patch': {'path': f'update/update.rpf/dlc_patch/{dlc_name}'}})
    print(f)

In [ ]:
# Scan for all content.xml, weapon<...>.meta, pickups.meta files in all dlc archives.
for dlc in dlcs.keys():
    for installment in ['base', 'patch']:
        if (installment == 'base') or ('patch' in dlcs[dlc].keys()):
            dlcs[dlc][installment].update({'weapons': [], 'pickups': [], 'content': []})
            # Scan content.xml
            if os.path.isfile(f'{ROOT_PATH}/{dlcs[dlc][installment]['path']}/content.xml'):
                dlcs[dlc][installment]['content'] = [f"{dlcs[dlc][installment]['path']}/content.xml"]
            # Scan weapon-.meta | Will need better filtering when working with more types of config
            path_folder = f"{ROOT_PATH}/{dlcs[dlc][installment]['path']}/common/data/ai"
            if os.path.isdir(path_folder):
                for f in os.listdir(path_folder):
                    if ('weapon' in f) and (f.endswith('.meta')):
                        dlcs[dlc][installment]['weapons'].append(f'{dlcs[dlc][installment]['path']}/common/data/ai/{f}')
            # Scan pickups.meta
            if os.path.isfile(f'{ROOT_PATH}/{dlcs[dlc][installment]['path']}/common/data/pickups.meta'):
                dlcs[dlc][installment]['pickups'] = [f"{dlcs[dlc][installment]['path']}/common/data/pickups.meta"]

def print_dlcs(dlcs):
    bool_to_int = {True: 1, False: 0}
    for dlc, dlc_content in dlcs.items():
        print(f"{dlc.ljust(17)}    base     pickups: {bool_to_int[dlc_content['base']['pickups']['avail']]}    weapon: {dlc_content['base']['weapons']['files']}")
        if dlcs[dlc].get('patch'):
            print(f"{dlc.ljust(17)}    patch    pickups: {bool_to_int[dlc_content['patch']['pickups']['avail']]}    weapon: {dlc_content['patch']['weapons']['files']}")
# print_dlcs(dlcs)

# Export list of dlcs
with open(f'{PATH_OUTPUTS}/supplementary-docs/dlcs.yaml', 'w') as f:
    yaml.dump(data=dlcs, stream=f, indent=2, sort_keys=False)
print(f.name)

In [4]:
# Merge weapons data from all vanilla files
weaponmetaparser = WeaponMetaParser(template_source=f'{ROOT_PATH}/{dlcs['base']['base']['weapons'][0]}')
for dlc in dlcs.keys():
    for installment in ['base', 'patch']:
        if (installment == 'base') or ('patch' in dlcs[dlc].keys()):
            for f in dlcs[dlc][installment]['weapons']:
                new_weaponmeta = weaponmetaparser.parse_xmltodict(source=f'{ROOT_PATH}/{f}')
                weaponmetaparser.update(new=new_weaponmeta, metadata={'dlc': dlc, 'installment': installment, 'path': f})
weaponmetaparser.template['CWeaponInfoBlob']['Name'] = 'Merged 1.70'

In [ ]:
# Export merged weapons info to .meta
if True:
    f = f'{PATH_OUTPUTS}/merged/update/update.rpf/common/data/ai/weapons.meta'
    if not os.path.isdir(os.path.dirname(f)):
        os.makedirs(os.path.dirname(f))
    weapons_merged_meta = deepcopy(weaponmetaparser.template)
    weapons_merged_meta_xml = ET.ElementTree(element=ET.fromstring(text=xmltodict.unparse(
        input_dict=weapons_merged_meta,
        encoding='utf-8',
        pretty=True,
        indent=2,
    )))
    weapons_merged_meta_xml.write(
        file_or_filename=f,
        encoding="utf-8",
        xml_declaration=True,
    )
    print(f)

# Export logs from callbacks (list of weapons and weapons by dlc)
with open(f'{PATH_OUTPUTS}/supplementary-docs/ammos_by_dlc.yaml', 'w') as f:
    yaml.dump(data=weaponmetaparser.structure['Infos'][0]['callbacks'][0].data, stream=f, indent=2, sort_keys=False)
print(f.name)

with open(f'{PATH_OUTPUTS}/supplementary-docs/weapons_by_dlc.yaml', 'w') as f:
    yaml.dump(data=weaponmetaparser.structure['Infos'][1]['callbacks'][0].data, stream=f, indent=2, sort_keys=False)
print(f.name)

In [ ]:
# Merge pickups data from all vanilla files
pickupsmetaparser = PickupsMetaParser(template_source=f'{ROOT_PATH}/{dlcs['base']['base']['pickups'][0]}')
for dlc in dlcs.keys():
    for installment in ['base', 'patch']:
        if (installment == 'base') or ('patch' in dlcs[dlc].keys()):
            for f in dlcs[dlc][installment]['pickups']:
                new_pickups = pickupsmetaparser.parse_xmltodict(source=f'{ROOT_PATH}/{f}')
                pickupsmetaparser.update(new=new_pickups, metadata={'dlc': dlc, 'installment': installment, 'path': f})

# Export merged pickups info to .meta
if True:
    f = f'{PATH_OUTPUTS}/merged/update/update.rpf/common/data/pickups.meta'
    pickups_merged_meta = deepcopy(pickupsmetaparser.template)
    pickups_merged_meta_xml = ET.ElementTree(element=ET.fromstring(text=xmltodict.unparse(
        input_dict=pickups_merged_meta,
        encoding='utf-8',
        pretty=True,
        indent=2,
    )))
    pickups_merged_meta_xml.write(
        file_or_filename=f,
        encoding="utf-8",
        xml_declaration=True,
    )
    print(f)

    # Export logs from callbacks (list of weapons and weapons by dlc)
    with open(f'{PATH_OUTPUTS}/supplementary-docs/pickups_by_dlc.yaml', 'w') as f:
        yaml.dump(data=pickupsmetaparser.structure['pickupData']['callbacks'][0].data, stream=f, indent=2, sort_keys=False)
    print(f.name)

In [ ]:
# Convert content.xml to deactivate old weapon-.meta and pickups.meta
for dlc in dlcs.keys():
    if (dlc == 'base'):
        continue

    # Compose list of file to unload | Skip if no need to unload
    blacklist = []
    for installment in ['base', 'patch']:
        if (installment == 'patch') and (dlcs[dlc].get('patch') is None):
            continue
        for w in dlcs[dlc][installment]['weapons']:
            w_name = f'common/data/ai/{w.split(sep='common/data/ai/')[-1]}'
            if w_name not in blacklist:
                blacklist.append(w_name)
        for p in dlcs[dlc][installment]['pickups']:
            p_name = f'common/data/{p.split(sep='common/data/')[-1]}'
            if p_name not in blacklist:
                blacklist.append(p_name)

    if blacklist == []:
        continue
    
    for installment in ['base', 'patch']:
        if (installment == 'patch') and (dlcs[dlc].get('patch') is None):
            continue
        
        if len(dlcs[dlc][installment]['content']) > 0:
            content_xml = ET.parse(f'{ROOT_PATH}/{dlcs[dlc][installment]["content"][0]}').getroot()
            for ccs in content_xml.find('contentChangeSets'):
                if 'AUTOGEN' not in ccs.find('changeSetName').text:
                    continue
                
                files_to_enable = ccs.find('filesToEnable')
                for fe in list(files_to_enable):
                    flag_unload = [fu in fe.text.lower() for fu in blacklist]
                    if any(flag_unload):
                        files_to_enable.remove(fe)
                        print(f'{dlc}-{installment}: {fe.text} unloaded')
        
        # Export content.xml to dlc_patch
        # ! Both installment 'base' and 'patch' are process, but then 'patch'
        #   can overwrite 'base'
        f = f'{PATH_OUTPUTS}/merged/update/update.rpf/dlc_patch/{dlc}/content.xml'
        if not os.path.isdir(os.path.dirname(f)):
            os.makedirs(os.path.dirname(f))
        ET.ElementTree(element=content_xml).write(
            file_or_filename=f,
            encoding="utf-8",
            xml_declaration=True,
        )
        print(f'{dlc}-{installment}: {f} saved')

In [ ]:
# Filter content.xml files per dlc_patch for uninstallation
flag_backup_content = {
    'to-delete': [],
    'to-patch': [],
}

for dlc in dlcs.keys():
    if (dlc == 'base'):
        continue
    
    if len(dlcs[dlc]['base']['content']) > 0:
        flag_backup_content['to-delete'].append(dlc)
        
        if (dlcs[dlc].get('patch') is not None):
            if len(dlcs[dlc]['patch']['content']) > 0:
                flag_backup_content['to-delete'].remove(dlc)
                flag_backup_content['to-patch'].append(dlc)

flag_backup_content['to-delete'].sort()
flag_backup_content['to-patch'].sort()

# Export logs from callbacks (list of weapons and weapons by dlc)
with open(f'{PATH_OUTPUTS}/uninstall-content-dlcpatch.yaml', 'w') as f:
    yaml.dump(data=flag_backup_content, stream=f, indent=2, sort_keys=False)
print(f.name)